Modify the Gradient Boosting scratch code in our lecture such that:
- Notice that we are still using max_depth = 1.  Attempt to tweak min_samples_split, max_depth for the regression and see whether we can achieve better mse on our boston data
- Notice that we only write scratch code for gradient boosting for regression, add some code so that it also works for binary classification.  Load the breast cancer data from sklearn and see that it works.
- Further change the code so that it works for multiclass classification.  Load the digits data from sklearn and see that it works
- Put everything into class

In [1]:
from scipy.special import expit
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
import numpy as np

class GradientBoosting:
    def __init__(self, S=5, learning_rate=1, max_depth = 1, 
                 min_samples_split = 2,
                 regression=True, tol=1e-4):
        self.S = S
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.regression=regression
            
        #initialize regression trees
        tree_params = {'max_depth': self.max_depth,
                      'min_samples_split': self.min_samples_split}
        self.models = [DecisionTreeRegressor(**tree_params) for _ in range(S)]
        
        if regression == True:
            # if GBregressor use Regressor as the first model
            first_model = DummyRegressor(strategy='mean')
        else :
            # if GBclassifier use Classifier as the first model
            first_model = DummyClassifier(strategy='most_frequent')
            
        self.models.insert(0, first_model)
        
    def grad(self, y, h):
        return y - h
    
    def fit(self, X, y):  #<----X_train
        
        #fit the first model
        self.models[0].fit(X, y)
        
        for i in range(self.S):
            #predict
            yhat = self.predict(X, self.models[:i+1], with_argmax=False)
            
            #get the gradient
            gradient = self.grad(y, yhat)
            
            #fit the next model with gradient
            self.models[i+1].fit(X, gradient)
    
    def predict(self, X, models=None, with_argmax=True):
        if models is None:
            models = self.models
        h0 = models[0].predict(X)  #first use the dummy model
        boosting = sum(self.learning_rate * model.predict(X) for model in models[1:])
        yhat = h0 + boosting
        if not self.regression:
            #turn into probability using softmax
            yhat = np.exp(yhat) / np.sum(np.exp(yhat), axis=1, keepdims=True)
            if with_argmax:
                yhat = np.argmax(yhat, axis=1)
        return yhat



In [2]:
# Regression

from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error

X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                        test_size=0.3, random_state=42)

model = GradientBoosting(S=200, learning_rate=0.1, max_depth = 3, 
                 min_samples_split = 2,
                 regression=True, tol=1e-4)
model.fit(X_train, y_train)
yhat = model.predict(X_test)

#print metrics
print("MSE: ", mean_squared_error(y_test, yhat))

n_estimators = 200

#=====SKlearn========
#Compare to sklearn: ls is the same as our mse
sklearn_model = GradientBoostingRegressor(
    n_estimators=n_estimators,
    learning_rate = 0.1,
    max_depth=3,
    loss='ls'
)

yhat_sk = sklearn_model.fit(X_train, y_train).predict(X_test)
print("Sklearn MSE: ", mean_squared_error(y_test, yhat_sk))

MSE:  7.815870385313655
Sklearn MSE:  7.773228216621459


In [3]:
# Binary classification

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

X, y = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.3, random_state=42)
y_train_encoded = np.zeros((y_train.shape[0], len(set(y))))
for each_class in range(len(set(y))):
    cond = y_train==each_class
    y_train_encoded[np.where(cond), each_class] = 1

model = GradientBoosting(S=200, learning_rate=0.1, max_depth = 3, 
                 min_samples_split = 2,
                 regression=False)
model.fit(X_train, y_train_encoded)
yhat = model.predict(X_test)

# #print metrics
print("Our accuracy: ", accuracy_score(y_test, yhat))

#=====SKlearn========
#Compare to sklearn: ls is the same as our accuracy
sklearn_model = GradientBoostingClassifier(
    n_estimators=n_estimators,
    learning_rate = 0.1,
    max_depth=1
)

yhat_sk = sklearn_model.fit(X_train, y_train).predict(X_test)
print("Sklearn accuracy: ", accuracy_score(y_test, yhat_sk))

Our accuracy:  0.9649122807017544
Sklearn accuracy:  0.9649122807017544


In [4]:
# Multiclass classification

from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

X, y = load_digits(return_X_y=True)

X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.3, random_state=42)
y_train_encoded = np.zeros((y_train.shape[0], len(set(y))))
for each_class in range(len(set(y))):
    cond = y_train==each_class
    y_train_encoded[np.where(cond), each_class] = 1

model = GradientBoosting(S=200, learning_rate=0.1, max_depth = 3, 
                 min_samples_split = 2,
                 regression=False)
model.fit(X_train, y_train_encoded)
yhat = model.predict(X_test)

# #print metrics
print("Our accuracy: ", accuracy_score(y_test, yhat))

#=====SKlearn========
#Compare to sklearn: ls is the same as our accuracy
sklearn_model = GradientBoostingClassifier(
    n_estimators=n_estimators,
    learning_rate = 0.1,
    max_depth=1
)

yhat_sk = sklearn_model.fit(X_train, y_train).predict(X_test)
print("Sklearn accuracy: ", accuracy_score(y_test, yhat_sk))

Our accuracy:  0.9296296296296296
Sklearn accuracy:  0.9481481481481482
